In [7]:
import numpy as np
import matplotlib.pyplot as plt
from astropy import constants as const
from astropy import units as u

c = const.c.value
G = const.G.value
M_sun = const.M_sun.value
M_earth = const.M_earth.value

In [8]:
def rel_len(x_1, x_2):
    r_vec = x_1 - x_2
    r = np.linalg.norm(r_vec)
    return r

def findrCM(x_1, x_2, m_1, m_2):
    M = m_1 + m_2
    r_cm = (m_1*x_1 + m_2*x_2)/M
    return r_cm

# def findvCM(s):
#     x_1, x_2, v_1, v_2, m_1, m_2 = s
#     M = m_1 + m_2
#     v_cm = (m_1*v_1 + m_2*v_2)/M
#     return v_cm

def findR_S(m_1, m_2, DL = False):
    if DL == True:
        c = 1
        G = 1
    else:
        c = const.c.value
        G = const.G.value
    R_S = 2 * (m_1 + m_2) * G/c**2
    return R_S

# def findp(s):
#     x_1, x_2, v_1, v_2, m_1, m_2 = s
#     # v_cm = findvCM(s)
#     # p_1 = m_1*(v_1 - v_cm)
#     # p_2 = m_2*(v_2 - v_cm)
#     p_1 = m_1*v_1
#     p_2 = m_2*v_2
#     return p_1, p_2

def findL(s):
    x_1, x_2, v_1, v_2, m_1, m_2 = s
    p_1, p_2 = findp(s)
    # r_cm = findrCM(x_1, x_2, m_1, m_2)
    # L_1 = np.cross(x_1 - r_cm, p_1)
    # L_2 = np.cross(x_2 - r_cm, p_2)
    L_1 = np.cross(x_1, p_1)
    L_2 = np.cross(x_2, p_2)
    return L_1, L_2

def findb(s):
    x_1, x_2, v_1, v_2, m_1, m_2 = s
    p_1, p_2 = findp(s)
    L_1, L_2 = findL(s)
    p_vec = p_1 - p_2
    L_vec = L_1 - L_2
    p = np.linalg.norm(p_vec)
    L = np.linalg.norm(L_vec)
    # b_vec = L_vec/p_vec
    b = L/p
    return b

def findE(s):
    x_1, x_2, v_1, v_2, m_1, m_2 = s
    r_vec = x_1 - x_2
    r = np.linalg.norm(r_vec)
    n = r_vec/r
    
    p_1, p_2 = findp(s)
    E_1PN = 1/c**2*(-1/8*np.linalg.norm(p_1)**2/m_1**3  +  1/8*G*m_1*m_2/r*(  -12*np.linalg.norm(p_1)**2/m_1**2  +  14*np.dot(p_1, p_2)/(m_1*m_2)  +  2*np.dot(n, p_1)*np.dot(n, p_2)/(m_1*m_2)  )  +1/4*G*m_1*m_2/r*G*(m_1 + m_2)/r    +     -1/8*np.linalg.norm(p_2)**2/m_2**3  +  1/8*G*m_2*m_1/r*(  -12*np.linalg.norm(p_2)**2/m_2**2  +  14*np.dot(p_2, p_1)/(m_2*m_1)  +  2*np.dot(-n, p_2)*np.dot(-n, p_1)/(m_2*m_1)  )  +1/4*G*m_2*m_1/r*G*(m_2 + m_1)/r  )
    return E_1PN

def γ(v):
    return np.sqrt(1 - np.dot(v,v)/c**2)**(-1)

In [ ]:
def fourdot(V_1, V_2):
    return np.array([V_1[0]*V_2[0] - V_1[1]*V_2[1] - V_1[2]*V_2[2]])

In [9]:
def SItoDL(s, t_max, dt):
    c = const.c.value
    G = const.G.value

    x_1, x_2, v_1, v_2, m_1, m_2 = s
    R_S = findR_S(m_1, m_2, DL = False)

    X_1, X_2 = x_1/R_S, x_2/R_S
    V_1, V_2 = v_1/c, v_2/c
    M_1, M_2 = m_1 * G/(c**2*R_S), m_2 * G/(c**2*R_S)

    S = X_1, X_2, V_1, V_2, M_1, M_2
    T_max, dT = t_max * c/R_S, dt * c/R_S

    return S, T_max, dT, R_S

# def DLtoSI(S, t_max, dt):
#     c = const.c.value
#     G = const.G.value

#     X_1, X_2, V_1, V_2, M_1, M_2 = S
#     R_S = findR_S(m_1, m_2, DL = True)

#     x_1, x_2 = X_1*R_S, X_2*R_S
#     v_1, v_2 = V_1*c, V_2*c
#     m_1, m_2 = M_1 /( G/(c**2*R_S)), M_2 /( G/(c**2*R_S))

#     S = x_1, x_2, v_1, v_2, m_1, m_2
#     t_max, dt = T_max / (c/R_S), dT / (c/R_S)

#     return s, t_max, dt, R_S

In [1]:
def accelerationN_OLD(x_1, x_2, m_1, m_2, DL):
    if DL == True:
        G = 1
    else:
        G = const.G.value
    
    r_vec = x_1 - x_2
    r = rel_len(x_1, x_2)
    n = r_vec/r

    a_1 = - G*m_2*n/r**2
    a_2 =   G*m_1*n/r**2
    return np.array([a_1, a_2])

def accelerationN(x_1, x_2, v_1, v_2, m_1, m_2):
    c = const.c.value
    G = const.G.value

    r_vec = x_1 - x_2
    r = np.sqrt(np.dot(r_vec, r_vec))

    p_vec = m_1*v_1
    p = np.sqrt(np.dot(p_vec, p_vec))

    E_1 = np.sqrt((m_1*c**2)**2 + (p*c)**2)
    E_2 = np.sqrt((m_2*c**2)**2 + (p*c)**2)

    def q_σ_dot(p_σ):
        return p_σ*(1/m_1 + 1/m_2)

    def p_σ_dot(x_i):
        return - G*m_1*m_2/r**3*x_i

    def a_q_σ(x_i):
        return -G*m_1*m_2/r**3*(1/m_1 + 1/m_2)*x_i

    x1_dot = np.array([q_σ_dot(p_vec[0]), q_σ_dot(p_vec[1])])
    px1_dot = np.array([p_σ_dot(x_1[0]), p_σ_dot(x_1[1])])

    x2_dot = np.array([q_σ_dot(-p_vec[0]), q_σ_dot(-p_vec[1])])
    px2_dot = np.array([p_σ_dot(x_2[0]), p_σ_dot(x_2[1])])

    a_1 = np.array([a_q_σ(x_1[0]), a_q_σ(x_1[1])])
    a_2 = np.array([a_q_σ(x_2[0]), a_q_σ(x_2[1])])
    
    return np.array([a_1, a_2])


def accelerationEIH(x_1, x_2, v_1, v_2, m_1, m_2, DL, α, β):
    if DL == True:
        G, c = 1, 1
    else:
        c = const.c.value
        G = const.G.value
    
    r_vec = x_1 - x_2
    r = rel_len(x_1, x_2)
    v_vec = v_1 - v_2
    n = r_vec/r

    a_1 = - β * G*m_2/r**2*n + α * 1/c**2*((5*G**2*m_1*m_2/r**3 + 4*G**2*m_2**2/r**3 + G*m_2/r**2*(3/2*np.dot(n, v_2)**2 - np.dot(v_1, v_1) + 4*np.dot(v_1, v_2) - 2*np.dot(v_2, v_2)))*n + G*m_2/r**2 *(4*np.dot(n, v_1) - 3*np.dot(n, v_2))*v_vec)
    a_2 = - β * G*m_1/r**2*(-n) + α * 1/c**2*((5*G**2*m_2*m_1/r**3 + 4*G**2*m_1**2/r**3 + G*m_1/r**2*(3/2*np.dot((-n), v_1)**2 - np.dot(v_2, v_2) + 4*np.dot(v_2, v_1) - 2*np.dot(v_1, v_1)))*(-n) + G*m_1/r**2 *(4*np.dot((-n), v_2) - 3*np.dot((-n), v_1))*(-v_vec))
    return np.array([a_1, a_2])

def accelerationDatF(x_1, x_2, v_1, v_2, m_1, m_2, DL):
    if DL == True:
        G, c = 1, 1
    else:
        c = const.c.value
        G = const.G.value
    
    r_vec = x_1 - x_2
    r = rel_len(x_1, x_2)
    v_vec = v_1 - v_2

    l = np.cross(r_vec, v_vec)

    a_1 = - G*m_2/r**3*(1 + 3*l**2/(c**2 * r**2))*r_vec
    a_2 =   G*m_1/r**3*(1 + 3*l**2/(c**2 * r**2))*r_vec
    return np.array([a_1, a_2])



def accelerationPM(x_1, x_2, P_1, P_2, m_1, m_2):
    P = np.array([P_1, P_2])
    E_1 = P_1[0]
    E_2 = P_2[0]
    Es = np.array([E_1, E_2])

    def q_dot(i, σ):
        j = i - 1
        return p(i, σ)/E(i) - G/r * (4*fourdot(P_1, P_2)/(E(1)*E(2)) * (E(j)/E(i)*p(i, σ) - p(j, σ)) + (m_1**2*m_2**2 - 2*fourdot(P_1,P_2)**2)/(E(1)*E(2))**2 * E(j)/E(i)*p(i, σ))

    def E(i):
        return Es[i-1]

    def E_dot(i):
        (p(i,1)*p_dot(i,1) + p(i,2)*p_dot(i,2))/E(i)

    def p(i, σ):
        return P[i-1,σ]

    def p_dot(i, σ):
        return G*r_vec[σ]/r**3 * (m_1**2*m_2**2 - 2*fourdot(P_1, P_2)**2)/(E_1*E_2)
    
    def P_dot(i):
        return np.array([E_dot(i), p_dot(i, 1),  p_dot(i, 2)])

    def a(i, σ):
        j = i - 1
        return p_dot(i, σ)/E(i) - p(i, σ)*E_dot(i)/E(i)**2 + G*r_dot*(4*fourdot(P_1, P_2)/(E(1)*E(2)) * (E(j)/E(i)*p(i, σ) - p(j, σ)) + (m_1**2*m_2**2 - 2*fourdot(P_1,P_2)**2)/(E(1)*E(2))**2 * E(j)/E(i)*p(i, σ))        - G/r * (4*((fourdot(P_dot(1), P_2) + fourdot(P_dot(2), P_1))/(E(1)*E(2)) - fourdot(P_1, P_2)/(E(1)*E(2))**2*(E_dot(1)*E(2) + E(1)*E_dot(2)))*(E(j)/E(i)*p(i, σ) - p(j, σ)) + 4 * fourdot(P_1, P_2)/(E(1)*E(2))*((E_dot(j)/E(i) - E(j)*E_dot(i)/E(i)**2)*p(i, σ) + E(j)/E(i)*p_dot(i, σ) - p_dot(j, σ))        - (4* fourdot(P_1, P_2)*(fourdot(P_dot(1), P_2) + fourdot(P_dot(2), P_1))/(E(1)*E(2))**2 - 2*(m_1**2*m_2**2 - 2*fourdot(P_1,P_2)**2)/(E(1)*E(2))**3*(E_dot(1)*E(2) + E(1)*E_dot(2)))*E(j)/E(i)*p(i, σ)        + (m_1**2*m_2**2 - 2*fourdot(P_1,P_2)**2)/(E(1)*E(2))**2*( (E_dot(j)/E(i) - E(j)*E_dot(i)/E(i)**2)*p(i, σ) + E(j)/E(i)*p_dot(i, σ)))
    
    c = const.c.value
    G = const.G.value

    r = rel_len(x_1, x_2)
    r_vec = x_1 - x_2
    r_vec = [0, r_vec[0], r_vec[1]]
    r_dot = (r_vec[0]*(q_dot(1, 1) - q_dot(2, 1)) + r_vec[1]*(q_dot(1, 2) - q_dot(2, 2)))/r

    a_1 = np.array([a(1, 1), a(1, 2)])
    a_2 = np.array([a(2, 1), a(2, 2)])
    
    return np.array([a_1, a_2])


def boost(v_1, v_2, a_1, a_2, dt):
    v_1 += a_1*dt
    v_2 += a_2*dt
    return np.array([v_1, v_2])

def move(x_1, x_2, v_1, v_2, dt):
    x_1 += v_1*dt
    x_2 += v_2*dt
    return np.array([x_1, x_2])

def boost2(v_1, v_2, a_1, a_2, dt, d_i):
    v_1 += d_i*a_1*dt
    v_2 += d_i*a_2*dt
    return np.array([v_1, v_2])

def move2(x_1, x_2, v_1, v_2, dt, c_i):
    x_1 += c_i*v_1*dt
    x_2 += c_i*v_2*dt
    return np.array([x_1, x_2])

IndentationError: expected an indented block (2758709348.py, line 99)

In [11]:
def run_model(s, t_max, dt, mode = 'PM'):
    
    x_1, x_2, v_1, v_2, m_1, m_2 = s
    a_1, a_2 = 0, 0

    pos1 = np.zeros((2, int(t_max/dt)))
    pos2 = np.zeros((2, int(t_max/dt)))
    pos_CM = np.zeros((2, int(t_max/dt)))

    for i in range(int(t_max/dt)):
        pos1[:,i] = x_1
        pos2[:,i] = x_2
        pos_CM[:,i] = findrCM(x_1, x_2, m_1, m_2)
        if mode == 'PM':
            a_1, a_2 = accelerationPM(x_1, x_2, v_1, v_2, m_1, m_2)
        elif mode == 'N':
            a_1, a_2 = accelerationN(x_1, x_2, v_1, v_2, m_1, m_2)
        else:
            print('Error')
            break
        v_1, v_2 = boost(v_1, v_2, a_1, a_2, dt)
        x_1, x_2 = move(x_1, x_2, v_1, v_2, dt)
    return pos1, pos2, pos_CM

def run_model2(s, t_max, dt, mode = 'N', DL=False, α=1, β=1):
    c1 = 1/(2*(2-2**(1/3)))
    c2 = (1-2**(1/3))/(2*(2-2**(1/3)))
    c3 = c2
    c4 = c1

    d1 = 1/(2-2**(1/3))
    d2 = - 2**(1/3)/(2-2**(1/3))
    d3 = d1
    d4 = 0

    const = np.array([[c1,d1],[c2,d2],[c3,d3],[c4,d4]])
    
    if DL == True:
        s, t_max, dt, R_S = SItoDL(s, t_max, dt)
    
    x_1, x_2, v_1, v_2, m_1, m_2 = s
    a_1, a_2 = 0, 0

    R_S = findR_S(m_1, m_2)

    p_1, p_2 = findp(s)
    p_1_len, p_2_len = np.linalg.norm(p_1), np.linalg.norm(p_2)

    L_1, L_2 = findL(s)
    L_1_len, L_2_len = np.linalg.norm(L_1), np.linalg.norm(L_2)

    b = findb(s)
    E = findE(s)

    pos1 = np.zeros((2, int(t_max/dt)))
    pos2 = np.zeros((2, int(t_max/dt)))
    pos_CM = np.zeros((2, int(t_max/dt)))

    for i in range(int(t_max/dt)):
        pos1[:,i] = x_1
        pos2[:,i] = x_2
        pos_CM[:,i] = findrCM(x_1, x_2, m_1, m_2)
        if mode == 'EIH':
            a_1, a_2 = accelerationEIH(x_1, x_2, v_1, v_2, m_1, m_2, DL, α, β)
        elif mode == 'DatF':
            a_1, a_2 = accelerationDatF(x_1, x_2, v_1, v_2, m_1, m_2, DL)
        elif mode == 'N':
            a_1, a_2 = accelerationN(x_1, x_2, m_1, m_2, DL)
        else:
            print('Error')
            break
        for j,k in const:
            v_1, v_2 = boost2(v_1, v_2, a_1, a_2, dt, k)
            x_1, x_2 = move2(x_1, x_2, v_1, v_2, dt, j)
        
    end_pos = np.array([pos1[0][-1], pos1[1][-1] - b])
    θ_rad =  np.arctan2(end_pos[1],end_pos[0])
    θ_ang = int(θ_rad*180/np.pi)

    parameters = np.array([E, p_1_len, p_2_len, L_1_len, L_2_len, R_S, b, θ_ang], dtype='object')
    return pos1, pos2, pos_CM, parameters

In [12]:
def format_value(value, decimals):
    """ 
    Checks the type of a variable and formats it accordingly.
    Floats has 'decimals' number of decimals.
    """
    
    if isinstance(value, float):
        return f'{value:.{decimals}e}'
    elif isinstance(value, int):
        return f'{value:d}'
    else:
        return f'{value}'


def values_to_string(values, decimals):
    """ 
    Loops over all elements of 'values' and returns list of strings
    with proper formating according to the function 'format_value'. 
    """
    
    res = []
    for value in values:
        if isinstance(value, list):
            tmp = [format_value(val, decimals) for val in value]
            res.append(f'{tmp[0]} +/- {tmp[1]}')
        else:
            res.append(format_value(value, decimals))
    return res

def len_of_longest_string(s):
    """ Returns the length of the longest string in a list of strings """
    return len(max(s, key=len))

def nice_string_output(d, extra_spacing=5, decimals=3):
    """ 
    Takes a dictionary d consisting of names and values to be properly formatted.
    Makes sure that the distance between the names and the values in the printed
    output has a minimum distance of 'extra_spacing'. One can change the number
    of decimals using the 'decimals' keyword.  
    """
    
    names = d.keys()
    max_names = len_of_longest_string(names)
    
    values = values_to_string(d.values(), decimals=decimals)
    max_values = len_of_longest_string(values)
    
    string = ""
    for name, value in zip(names, values):
        spacing = extra_spacing + max_values + max_names - len(name) - 1 
        string += "{name:s} {value:>{spacing}} \n".format(name=name, value=value, spacing=spacing)
    return string[:-2]


def add_text_to_ax(x_coord, y_coord, string, ax, fontsize=12, color='k'):
    """ Shortcut to add text to an ax with proper font. Relative coords."""
    ax.text(x_coord, y_coord, string, ha='left', va='bottom', family='monospace', fontsize=fontsize,
            transform=ax.transAxes, verticalalignment='top', color=color)
    return None

def findMaxPos(pos):
    xmax, xmin, ymax, ymin = max(pos[0]), min(pos[0]), max(pos[1]), min(pos[1])
    return xmax, xmin, ymax, ymin

def plotLims(pos):
    start_pos = np.array([pos[0][0], pos[1][0]])
    end_pos = np.array([pos[0][-1], pos[1][-1]])

    xmax, xmin, ymax, ymin = findMaxPos(pos)
    max_pos = max(max(abs(start_pos)),max(abs(end_pos)))

    xlim = (xmin-0.1*max_pos, xmax+0.1*max_pos)
    ylim = (ymin-0.1*max_pos, ymax+0.1*max_pos)
    return xlim, ylim

def plotLimsTwoBody(pos1, pos2):
    start_pos1 = np.array([pos1[0][0], pos1[1][0]])
    end_pos1 = np.array([pos1[0][-1], pos1[1][-1]])
    start_pos2 = np.array([pos2[0][0], pos2[1][0]])
    end_pos2 = np.array([pos2[0][-1], pos2[1][-1]])

    x1max, x1min, y1max, y1min = findMaxPos(pos1)
    x2max, x2min, y2max, y2min = findMaxPos(pos2)

    xmax = max(x1max, x2max)
    xmin = min(x1min, x2min)
    ymax = max(y1max, y2max)
    ymin = min(y1min, y2min)

    max_pos1 = max(max(abs(start_pos1)),max(abs(end_pos1)))
    max_pos2 = max(max(abs(start_pos2)),max(abs(end_pos2)))
    max_pos = max(max_pos1, max_pos2)

    xlim = (xmin-0.1*max_pos, xmax+0.1*max_pos)
    ylim = (ymin-0.1*max_pos, ymax+0.1*max_pos)
    return xlim, ylim

def orbPlotter(pos1, pos2, posCM, parameters = np.zeros(8), aspect = 1, filename='', CM = True, save = False, show = True, DL = False, figsize=(8,8)):
    x_1 = pos1
    x_2 = pos2 
    x_cm = posCM
    if CM == True:
        x_1 = x_1 - x_cm
        x_2 = x_2 - x_cm
        x_cm = np.zeros_like(x_1)

    fig, ax = plt.subplots(figsize=figsize)
    xlim, ylim = plotLimsTwoBody(x_1, x_2)
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)

    # ax.plot(x_cm[0], x_cm[1], 'g:', label = 'CM', lw=0.4)
    ax.plot(x_cm[0], x_cm[1], 'g:', lw=0.5)
    ax.plot(x_cm[0][0], x_cm[1][0], 'g.',markersize=8)
    ax.plot(x_cm[0][-1], x_cm[1][-1], 'g.',markersize=8)

    # ax.plot(x_1[0], x_1[1],'b', label = 'm_1', lw=0.75)
    ax.plot(x_1[0], x_1[1],'b', lw=1)
    ax.plot(x_1[0][0], x_1[1][0], 'bo', label = 'm_1 start', markersize=15)
    ax.plot(x_1[0][-1], x_1[1][-1], 'b.', label = 'm_1 stop',markersize=15)

    # ax.plot(x_2[0], x_2[1],'r', label = 'm_2', lw=0.75)
    ax.plot(x_2[0], x_2[1],'r', lw=1)
    ax.plot(x_2[0][0], x_2[1][0], 'ro', label = 'm_2 start', markersize=15)
    ax.plot(x_2[0][-1], x_2[1][-1], 'r.', label = 'm_2 stop',markersize=15)

    if DL == True:
        ax.set_xlabel('$R_S$', fontsize = 15)
        ax.set_ylabel('$R_S$', fontsize = 15)
    else:
        ax.set_xlabel('$x \ [\mathrm{m}]$', fontsize = 15)
        ax.set_ylabel('$y \ [\mathrm{m}]$', fontsize = 15)  
    ax.grid(c='grey', alpha=0.2, ls ='--')
    ax.set_aspect(aspect)
    ax.set_title(f'{filename}', fontsize = 20)
    ax.legend(facecolor='grey', fontsize = 12, bbox_to_anchor=(1.01,1), loc='upper left')

    d = {
         'E:':      parameters[0],
         '|p_1|:':  parameters[1],
         '|p_2|:':  parameters[2],
         '|L_1|:':  parameters[3],
         '|L_2|:':  parameters[4],
         'R_S:':    parameters[5],
         'b:':      parameters[6],
         'θ:':      parameters[7],
        }

    text = nice_string_output(d, extra_spacing=3, decimals=2)
    add_text_to_ax(1.04, 0, text, ax, fontsize=12)

    plt.title(f'{filename}', fontsize=25, y=1.08)
    fig.patch.set_facecolor('white')
    fig.tight_layout()
    if show == False:
        plt.close(fig)
    if save == True:
        fig.savefig(f'./Plots/{filename}.png', dpi=300, transparent = False)
    return

def orbPlotter_lims(pos1, pos2, posCM, xlim, ylim, parameters = np.zeros(8), aspect = 1, filename='', CM = False, save = False, show = True, DL = False, figsize=(8,8)):
    x_1 = pos1
    x_2 = pos2 
    x_cm = posCM
    if CM == True:
        x_1 = x_1 - x_cm
        x_2 = x_2 - x_cm
        x_cm = np.zeros_like(x_1)

    fig, ax = plt.subplots(figsize=figsize)
    # xlim, ylim = plotLimsTwoBody(x_1, x_2)
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)

    ax.plot(x_cm[0], x_cm[1], 'g:',label = 'CM')
    ax.plot(x_cm[0][0], x_cm[1][0], 'g.',markersize=8)
    ax.plot(x_cm[0][-1], x_cm[1][-1], 'g.',markersize=8)

    ax.plot(x_1[0], x_1[1],'b', label = 'm_1')
    ax.plot(x_1[0][0], x_1[1][0], 'bo', label = 'm_1 start', markersize=15)
    ax.plot(x_1[0][-1], x_1[1][-1], 'b.', label = 'm_1 stop',markersize=15)

    ax.plot(x_2[0], x_2[1],'r', label = 'm_2')
    ax.plot(x_2[0][0], x_2[1][0], 'ro', label = 'm_2 start', markersize=15)
    ax.plot(x_2[0][-1], x_2[1][-1], 'r.', label = 'm_2 stop',markersize=15)

    if DL == True:
        ax.set_xlabel('$R_S$', fontsize = 15)
        ax.set_ylabel('$R_S$', fontsize = 15)
    else:
        ax.set_xlabel('$x \ [\mathrm{m}]$', fontsize = 15)
        ax.set_ylabel('$y \ [\mathrm{m}]$', fontsize = 15)  
    ax.grid(c='grey', alpha=0.2, ls ='--')
    ax.set_aspect(aspect)
    ax.set_title(f'{filename}', fontsize = 20)
    ax.legend(facecolor='grey', fontsize = 12, bbox_to_anchor=(1.01,1), loc='upper left')

    d = {
         'E:':      parameters[0],
         '|p_1|:':  parameters[1],
         '|p_2|:':  parameters[2],
         '|L_1|:':  parameters[3],
         '|L_2|:':  parameters[4],
         'R_S:':    parameters[5],
         'b:':      parameters[6],
         'θ:':      parameters[7],
        }

    text = nice_string_output(d, extra_spacing=3, decimals=2)
    add_text_to_ax(1.04, 0, text, ax, fontsize=12)

    plt.title(f'{filename}', fontsize=25, y=1.08)
    fig.patch.set_facecolor('white')
    fig.tight_layout()
    if show == False:
        plt.close(fig)
    if save == True:
        fig.savefig(f'./Plots/{filename}.png', dpi=300, transparent = False)
    return

def prettyOrbPlotter(pos1, pos2, posCM, parameters, aspect = 1, filename='', CM = False, save = False, show = True, figsize=(11,11)):
    x_1 = pos1
    x_2 = pos2 
    x_cm = posCM
    if CM == True:
        x_1 = x_1 - x_cm
        x_2 = x_2 - x_cm
        x_cm = 0


    fig, ax = plt.subplots(figsize=figsize)
    xlim, ylim = plotLimsTwoBody(x_1, x_2)
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)

    # ax.plot(x_cm[0], x_cm[1], 'g:',label = 'CM')
    # ax.plot(x_cm[0][0], x_cm[1][0], 'g.',markersize=8)
    # ax.plot(x_cm[0][-1], x_cm[1][-1], 'g.',markersize=8)

    ax.plot(x_1[0], x_1[1],'b', label = 'm_1')
    ax.plot(x_1[0][0], x_1[1][0], 'b.', label = 'm_1 start', markersize=15)
    ax.plot(x_1[0][-1], x_1[1][-1], 'bo', label = 'm_1 stop',markersize=15)

    ax.plot(x_2[0], x_2[1],'r', label = 'm_2')
    ax.plot(x_2[0][0], x_2[1][0], 'r.', label = 'm_2 start', markersize=15)
    ax.plot(x_2[0][-1], x_2[1][-1], 'ro', label = 'm_2 stop',markersize=15)

    # ax.set_xlabel('$R_S$', fontsize = 15)
    # ax.set_ylabel('$R_S$', fontsize = 15)
    # ax.grid(c='grey', alpha=0.2, ls ='--')
    ax.set_aspect(aspect)
    # ax.set_title(f'{filename}', fontsize = 20)
    # ax.legend(facecolor='grey',loc='upper right', fontsize = 12)
    fig.patch.set_facecolor('white')
    fig.tight_layout()
    plt.tick_params(left = False, right = False , labelleft = False ,
                labelbottom = False, bottom = False)
                
    if show == False:
        plt.close(fig)
    if save == True:
        fig.savefig(f'./Plots/Pretty/{filename}_pretty.png', dpi=300, transparent = False)
    return